In [2]:
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
import pydicom as dicom
import cv2

In [3]:
# set parameters here
savepath = 'data'
seed = 0
np.random.seed(seed) # Reset the seed so all runs are the same.
random.seed(seed)
MAXVAL = 255  # Range [0 255]

# path to covid-19 dataset from https://github.com/ieee8023/covid-chestxray-dataset
imgpath = '../covid-chestxray-dataset/images' 
csvpath = '../covid-chestxray-dataset/metadata.csv'

# path to https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
kaggle_datapath = 'rsna-pneumonia-detection-challenge'
kaggle_csvname = 'stage_2_detailed_class_info.csv' # get all the normal from here
kaggle_csvname2 = 'stage_2_train_labels.csv' # get all the 1s from here since 1 indicate pneumonia
kaggle_imgpath = 'stage_2_train_images'

# parameters for COVIDx dataset
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

mapping = dict()
mapping['COVID-19'] = 'COVID-19'
mapping['SARS'] = 'pneumonia'
mapping['MERS'] = 'pneumonia'
mapping['Streptococcus'] = 'pneumonia'
mapping['Normal'] = 'normal'
mapping['Lung Opacity'] = 'pneumonia'
mapping['1'] = 'pneumonia'

# train/test split config
split = 0.1

In [4]:
# adapted from https://github.com/mlmed/torchxrayvision/blob/master/torchxrayvision/datasets.py#L814
csv = pd.read_csv(csvpath, nrows=None)
idx_pa = csv["view"] == "PA"  # Keep only the PA view
csv = csv[idx_pa]

pneumonias = ["COVID-19", "SARS", "MERS", "ARDS", "Streptococcus"]
pathologies = ["Pneumonia","Viral Pneumonia", "Bacterial Pneumonia", "No Finding"] + pneumonias
pathologies = sorted(pathologies)

In [5]:
print (csv)

     patientid  offset  sex   age       finding survival intubated  \
0            2     0.0    M  65.0      COVID-19        Y       NaN   
1            2     3.0    M  65.0      COVID-19        Y       NaN   
2            2     5.0    M  65.0      COVID-19        Y       NaN   
3            2     6.0    M  65.0      COVID-19        Y       NaN   
4            4     0.0    F  52.0      COVID-19      NaN       NaN   
5            4     5.0    F  52.0      COVID-19      NaN       NaN   
6            5     NaN  NaN   NaN          ARDS      NaN         Y   
7            6     0.0  NaN   NaN      COVID-19      NaN         Y   
8            6     4.0  NaN   NaN      COVID-19      NaN         Y   
9            3     4.0    M  74.0          SARS        N       NaN   
10           3     9.0    M  74.0          SARS        N       NaN   
11           3    10.0    M  74.0          SARS        N       NaN   
12           7     7.0    F  29.0          SARS        Y       NaN   
13           7    12

In [6]:
# get non-COVID19 viral, bacteria, and COVID-19 infections from covid-chestxray-dataset
# stored as patient id, image filename and label
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
for index, row in csv.iterrows():
    f = row['finding']
    if f in mapping:
        count[mapping[f]] += 1
        entry = [int(row['patientid']), row['filename'], mapping[f]]
        filename_label[mapping[f]].append(entry)

print('Data distribution from covid-chestxray-dataset:')
print(count)

Data distribution from covid-chestxray-dataset:
{'normal': 0, 'pneumonia': 24, 'COVID-19': 103}


In [7]:
# add covid-chestxray-dataset into COVIDx dataset
# since covid-chestxray-dataset doesn't have test dataset
# split into train/test by patientid
# for COVIDx:
# patient 8 is used as non-COVID19 viral test
# patient 31 is used as bacterial test
# patients 19, 20, 36, 42, 86 are used as COVID-19 viral test

for key in filename_label.keys():
    arr = np.array(filename_label[key])
    if arr.size == 0:
        continue
    # split by patients
    # num_diff_patients = len(np.unique(arr[:,0]))
    # num_test = max(1, round(split*num_diff_patients))
    # select num_test number of random patients
    if key == 'pneumonia':
        test_patients = ['8', '31']
    elif key == 'COVID-19':
        test_patients = ['19', '20', '36', '42', '86'] # random.sample(list(arr[:,0]), num_test)
    else: 
        test_patients = []
    print('Key: ', key)
    print('Test patients: ', test_patients)
    # go through all the patients
    for patient in arr:
        if patient[0] in test_patients:
            copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'test', patient[1]))
            test.append(patient)
            test_count[patient[2]] += 1
        else:
            copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'train', patient[1]))
            train.append(patient)
            train_count[patient[2]] += 1

print('test count: ', test_count)
print('train count: ', train_count)

Key:  pneumonia
Test patients:  ['8', '31']
Key:  COVID-19
Test patients:  ['19', '20', '36', '42', '86']
test count:  {'normal': 0, 'pneumonia': 5, 'COVID-19': 10}
train count:  {'normal': 0, 'pneumonia': 19, 'COVID-19': 93}


In [8]:
# path to https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
kaggle_datapath = '../rsna-pneumonia-detection-challenge'

In [9]:
# add normal and rest of pneumonia cases from https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
csv_normal = pd.read_csv(os.path.join(kaggle_datapath, kaggle_csvname), nrows=None)
csv_pneu = pd.read_csv(os.path.join(kaggle_datapath, kaggle_csvname2), nrows=None)
patients = {'normal': [], 'pneumonia': []}

for index, row in csv_normal.iterrows():
    if row['class'] == 'Normal':
        patients['normal'].append(row['patientId'])

for index, row in csv_pneu.iterrows():
    if int(row['Target']) == 1:
        patients['pneumonia'].append(row['patientId'])

for key in patients.keys():
    arr = np.array(patients[key])
    if arr.size == 0:
        continue
    # split by patients 
    # num_diff_patients = len(np.unique(arr))
    # num_test = max(1, round(split*num_diff_patients))
    test_patients = np.load('rsna_test_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
    # np.save('rsna_test_patients_{}.npy'.format(key), np.array(test_patients))
    for patient in arr:
        ds = dicom.dcmread(os.path.join(kaggle_datapath, kaggle_imgpath, patient + '.dcm'))
        pixel_array_numpy = ds.pixel_array
        imgname = patient + '.png'
        if patient in test_patients:
            cv2.imwrite(os.path.join(savepath, 'test', imgname), pixel_array_numpy)
            test.append([patient, imgname, key])
            test_count[key] += 1
        else:
            cv2.imwrite(os.path.join(savepath, 'train', imgname), pixel_array_numpy)
            train.append([patient, imgname, key])
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 885, 'pneumonia': 1058, 'COVID-19': 10}
train count:  {'normal': 7966, 'pneumonia': 8521, 'COVID-19': 93}


In [10]:
# final stats
print('Final stats')
print('Train count: ', train_count)
print('Test count: ', test_count)
print('Total length of train: ', len(train))
print('Total length of test: ', len(test))

Final stats
Train count:  {'normal': 7966, 'pneumonia': 8521, 'COVID-19': 93}
Test count:  {'normal': 885, 'pneumonia': 1058, 'COVID-19': 10}
Total length of train:  16580
Total length of test:  1953


In [11]:
# run this cell when adding in new covid data from covid-chextxray-dataset

# load in current train/test information
'''train_filepath = 'train_split_v2.txt'
test_filepath = 'test_split_v2.txt'
file = open(train_filepath, 'r') 
trainfiles = file.readlines() 
trainfiles = np.array([line.split() for line in trainfiles])
file = open(test_filepath, 'r')
testfiles = file.readlines()
testfiles = np.array([line.split() for line in testfiles])

# find the new entries in csv 
new_entries = []
for key in filename_label.keys():
    arr = np.array(filename_label[key])
    if arr.size == 0:
        continue
    for patient in arr:
        if patient[1] not in trainfiles and patient[1] not in testfiles:
            # if key is normal, bacteria or viral add to train folder
            if key in ['normal', 'pneumonia']:
                copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'train', patient[1]))
                train.append(patient)
                train_count[patient[2]] += 1
            else: 
                new_entries.append(patient)
new_entries = np.array(new_entries)

# 10% of new entries should go into in test
if new_entries.size > 0:
    num_diff_patients = len(np.unique(new_entries[:,0]))
    num_test = max(1, round(split*num_diff_patients))

    i = 0
    used_i = []
    # insert patients who are already in dataset into the respective train/test
    for patient in new_entries:
        if patient[0] in trainfiles:
            copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'train', patient[1]))
            train.append(patient)
            train_count[patient[2]] += 1
            used_i.append(i)
        elif patient[0] in testfiles:
            copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'test', patient[1]))
            test.append(patient)
            test_count[patient[2]] += 1
            used_i.append(i)
        i += 1
    # delete patients who have already been inserted
    new_entries = np.delete(new_entries, used_i, axis=0)

    # select num_test number of random patients
    test_patients = random.sample(list(new_entries[:,0]), num_test)
    print('test patients: ', test_patients)
    # add to respective train/test folders
    for patient in new_entries:
        if patient[0] in test_patients:
            copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'test', patient[1]))
            test.append(patient)
            test_count[patient[2]] += 1
        else:
            copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'train', patient[1]))
            train.append(patient)
            train_count[patient[2]] += 1

print('added test count: ', test_count)
print('added train count: ', train_count)'''

"train_filepath = 'train_split_v2.txt'\ntest_filepath = 'test_split_v2.txt'\nfile = open(train_filepath, 'r') \ntrainfiles = file.readlines() \ntrainfiles = np.array([line.split() for line in trainfiles])\nfile = open(test_filepath, 'r')\ntestfiles = file.readlines()\ntestfiles = np.array([line.split() for line in testfiles])\n\n# find the new entries in csv \nnew_entries = []\nfor key in filename_label.keys():\n    arr = np.array(filename_label[key])\n    if arr.size == 0:\n        continue\n    for patient in arr:\n        if patient[1] not in trainfiles and patient[1] not in testfiles:\n            # if key is normal, bacteria or viral add to train folder\n            if key in ['normal', 'pneumonia']:\n                copyfile(os.path.join(imgpath, patient[1]), os.path.join(savepath, 'train', patient[1]))\n                train.append(patient)\n                train_count[patient[2]] += 1\n            else: \n                new_entries.append(patient)\nnew_entries = np.array(new_e

In [12]:
# export to train and test csv
# format as patientid, filename, label, separated by a space
train_file = open("train_split_v2.txt","a") 
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open("test_split_v2.txt", "a")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()